In [1]:
import csv
import pandas as pd
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
X = pd.read_csv('train_values.csv')
Y = pd.read_csv('train_labels.csv')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert y_train and y_test to 1-dimensional arrays
y_train = y_train.values[:, 1]  # Select the first column
y_test = y_test.values[:, 1]    # Select the first column

[2 2 3 2 3]


In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier()  # o 'ovo' si prefieres one-vs-one

clf = LazyClassifier(verbose=1, ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

  0%|          | 0/32 [00:00<?, ?it/s]

  3%|▎         | 1/32 [00:17<08:51, 17.14s/it]

ROC AUC couldn't be calculated for AdaBoostClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'AdaBoostClassifier', 'Accuracy': 0.6242397498129353, 'Balanced Accuracy': np.float64(0.48078868084007076), 'ROC AUC': None, 'F1 Score': 0.5883848098440814, 'Time taken': 17.135428428649902}


  6%|▋         | 2/32 [00:35<09:02, 18.07s/it]

ROC AUC couldn't be calculated for BaggingClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'BaggingClassifier', 'Accuracy': 0.7066441549471423, 'Balanced Accuracy': np.float64(0.6370408127936686), 'ROC AUC': None, 'F1 Score': 0.7015793989053978, 'Time taken': 18.73131513595581}


...

ROC AUC couldn't be calculated for RidgeClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'RidgeClassifier', 'Accuracy': 0.5842750522821896, 'Balanced Accuracy': np.float64(0.4250084770767255), 'ROC AUC': None, 'F1 Score': 0.5148582936076008, 'Time taken': 0.7693080902099609}


 78%|███████▊  | 25/32 [05:22<00:53,  7.60s/it]

ROC AUC couldn't be calculated for RidgeClassifierCV
multi_class must be in ('ovo', 'ovr')
{'Model': 'RidgeClassifierCV', 'Accuracy': 0.5842558661575948, 'Balanced Accuracy': np.float64(0.42488304996138376), 'ROC AUC': None, 'F1 Score': 0.5147968541108785, 'Time taken': 1.5353145599365234}


 81%|████████▏ | 26/32 [05:28<00:42,  7.12s/it]

ROC AUC couldn't be calculated for SGDClassifier
multi_class must be in ('ovo', 'ovr')
{'Model': 'SGDClassifier', 'Accuracy': 0.5756221100899829, 'Balanced Accuracy': np.float64(0.41038661427255724), 'ROC AUC': None, 'F1 Score': 0.45293208206681707, 'Time taken': 5.98780632019043}
